<a href="https://colab.research.google.com/github/Antoine723/ProjetNLP/blob/main/ProjetNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf

from gdrive.MyDrive.ProjetNLP.rnn_utils import *

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [107]:
df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None)
dat=np.array([df.iloc[0:1000][5],df.iloc[800000:801000][5]])

# print(data)

# Traitement

In [108]:
def processing(data):
  if (isinstance(data,str)):
    #MINUSCULES
    data=data.lower()
    #TOKENIZATION
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    data=tokenizer.tokenize(data)

    #STOPWORDS
    data=[w for w in data if not w in list(nltk.corpus.stopwords.words('english'))]
    #STEMMING
    stemmer=LancasterStemmer()
    data=[stemmer.stem(w) for w in data]

    #LEMMATIZATION
    Word_Lemmatizer = WordNetLemmatizer()
    data=[Word_Lemmatizer.lemmatize(w) for w in data]

  else :
    #MINUSCULES
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=data[i][j].lower()

    #TOKENIZATION
    tokenizer = nltk.RegexpTokenizer(r'\w+')

    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=tokenizer.tokenize(data[i][j])

    #STOPWORDS
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[w for w in data[i][j] if not w in list(nltk.corpus.stopwords.words('english'))]
        

    #STEMMING
    stemmer=LancasterStemmer()
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[stemmer.stem(w) for w in data[i][j]]

    #LEMMATIZATION
    Word_Lemmatizer = WordNetLemmatizer()
    for i in range(len(data)):
      for j in range(len(data[i])):
        data[i][j]=[Word_Lemmatizer.lemmatize(w) for w in data[i][j]]
    # print(data)

  return data

In [111]:
dat=processing(dat)
# processing("This movie is a very bad one")

In [120]:
# print(data)

In [112]:
from collections import defaultdict
def defineVocab(data):
  v=set()
  for i in range(len(data)):
    for j in range(len(data[i])):
      for h in range(len(data[i][j])):
        v.add(data[i][j][h])

  vocab = defaultdict(lambda: 0)
  vocab['<PAD>'] = 1

  j=2
  for i in v:
    vocab[i]=j
    j+=1
  return vocab
  # print(len(vocab))

In [114]:
voca=defineVocab(dat)
print(voca)

defaultdict(<function defineVocab.<locals>.<lambda> at 0x7f7187910440>, {'<PAD>': 1, 'understand': 2, 'hatch': 3, 'ein': 4, 'officialtil': 5, 'colorofviol': 6, 'mang': 7, 'max': 8, 'tayswift': 9, 'ord': 10, 'miss_tattoo': 11, 'visit': 12, 'damnit': 13, 'swe': 14, 'idiot': 15, 'tragedy': 16, 'vivekg86': 17, 'blood': 18, 'bear': 19, 'smil': 20, 'ted': 21, 'endors': 22, 'stripper': 23, 'tejano': 24, 'jel': 25, 'bech': 26, 'radford': 27, '00': 28, 'woot': 29, 'wudnt': 30, 'baawwngggg': 31, 'planet': 32, 'deez': 33, 'gold': 34, 'westney': 35, 'chriswolfe2007': 36, 'apothecaryjer': 37, 'ym': 38, 'kisluvk': 39, 'overitnow': 40, 'plagu': 41, 'danzero': 42, 'julieebaby': 43, 'heah': 44, 'lorigam': 45, 'mi': 46, 'workin': 47, 'dizzy': 48, 'memb': 49, 'died': 50, 'as': 51, 'sidekick': 52, 'tshirt': 53, 'cut': 54, 'mad': 55, 'nationwideclass': 56, 'nd': 57, 'camer': 58, 'ind': 59, 'slushy': 60, 'yoeyfreshy': 61, 'co': 62, 'frog': 63, 'wow': 64, 'bluff': 65, 'behav': 66, 'tmrw': 67, 'useless': 68, 

In [53]:
# df=pd.DataFrame(vocab.items(),columns=["Word","Id"])
# df.to_csv("gdrive/MyDrive/ProjetNLP/vocab",index=False)

In [52]:
# pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab").set_index("Id").to_dict()

In [51]:
# vocab=pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab")
# print(vocab)

In [92]:
def transformWords(data,vocab):
  matrixData=[]
  maxLen=0
  if (isinstance(data,list)):
    for i in data:
      matrixData.append(vocab[i])
  else:
    for i in range(len(data)):
      for j in range(len(data[i])):
        if (maxLen < len(data[i][j])):
          maxLen= len(data[i][j])

    for i in range(len(data)):
      for j in range(len(data[i])):
        test=[]
        for k in range(maxLen):
          if (k >= len(data[i][j])):
            test.append(vocab['<PAD>'])
          else:
            test.append(vocab[data[i][j][k]])
        matrixData.append(test)
  return matrixData

In [ ]:
data=transformWords(data)
datas=np.array([df.iloc[1000:2000][5],df.iloc[801000:802000][5]])
datas=processing(datas)
datas=transformWords(datas)

In [ ]:
data=np.array(data)
datas=np.array(datas)

In [16]:
def buildModelLSTM(vocab):
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=1))
  model.add(tf.keras.layers.LSTM(128,activation="relu"))
  model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
  return model

In [8]:
def buildModelSimpleRNN():
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=1))
  model.add(tf.keras.layers.SimpleRNN(128,activation="relu"))
  model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
  return model

In [ ]:
print(data)

[[3247 2503 1010 ...    1    1    1]
 [1111 2347  791 ...    1    1    1]
 [3224 4596  394 ...    1    1    1]
 ...
 [1920 1034 3096 ...    1    1    1]
 [2084 2253 2078 ...    1    1    1]
 [ 107  667 1282 ...    1    1    1]]


In [124]:
# model=buildModel()
# def fitModel(model,trainSize): #trainSize = taille des données prises en compte pour le fit (ex : si on crée le modèle en lui donnant 20 phrases, trainSize=20)
#Attention, ici on a configuré le y_train pour qu'il y ait une part égale de commentaires positifs/négatifs
def fitModel(data,model,negSize,posSize):
  y_train=np.concatenate([np.zeros((negSize,1)),np.ones((posSize,1))])
  model.compile(loss='binary_crossentropy',optimizer="adam")
  model.fit(data,y_train,epochs=5)

In [125]:
import os
def final(sentence):
  threshold=0.5
  model_file="sentimentAnalysis"
  print(sentence)
  if (not os.path.isdir("gdrive/MyDrive/ProjetNLP/"+model_file)):
    df=pd.read_csv('gdrive/MyDrive/ProjetNLP/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",header=None)
    x_train=np.array([df.iloc[0:1000][5],df.iloc[800000:801000][5]])
    x_train=processing(x_train)
    vocab=defineVocab(x_train)
    df=pd.DataFrame(vocab.items(),columns=["Word","Id"])
    df.to_csv("gdrive/MyDrive/ProjetNLP/vocab")
    x_train=transformWords(x_train,vocab)
    x_train=np.array(x_train)
    model=buildModelLSTM(vocab)
    fitModel(x_train,model,1000,1000)
    model.save("gdrive/MyDrive/ProjetNLP/"+model_file)
    print("Pas de fichier")
  else:
    model=tf.keras.models.load_model("gdrive/MyDrive/ProjetNLP/"+model_file)
    vocab=pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab").set_index("Word").to_dict()["Id"]
    print("Fichier")
  sentence=processing(sentence)
  sentence=transformWords(sentence,vocab)
  sentence=np.array(sentence)
  if (len(sentence.shape)==1):
    if (model.predict(np.expand_dims(sentence,axis=0))<threshold):
      commentTypes="Negative"
    else:
      commentTypes="Positive"
  else:
    commentTypes=[]
    for i in range(len(sentence)):
      if (model.predict(np.expand_dims(sentence[i],axis=0))<threshold):
        commentTypes.append("Negative")
      else:
        commentTypes.append("Positive")
  return commentTypes




In [128]:
# print(final(data))
final("This movie is a very bad one")

This movie is a very bad one
Fichier


'Negative'

In [104]:
# pd.read_csv("gdrive/MyDrive/ProjetNLP/vocab").set_index("Word").to_dict()["Id"]

In [ ]:
model.predict(datas)

array([[1.0000000e+00],
       [0.0000000e+00],
       [1.9049644e-04],
       ...,
       [8.7085558e-05],
       [7.5491369e-01],
       [0.0000000e+00]], dtype=float32)

In [ ]:
from sklearn.metrics import f1_score
seuils=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
F1_score=[]
predict=model.predict(datas)
for seuil in seuils:
  predictions=[]
  for i in range(len(datas)):
    if (predict[i]<seuil):
      predictions.append(0)
    else:
      predictions.append(1)
  F1_score.append(f1_score(y_train,predictions))

In [ ]:
print(F1_score)
seuil=seuils[F1_score.index(max(F1_score))]
print(seuil)

[0.6180344478216819, 0.6037735849056604, 0.5934654525977503, 0.5851005981511691, 0.5833333333333334, 0.5790055248618784, 0.5757238307349667, 0.5699831365935919, 0.5636672325976231]
0.1


In [ ]:
np.expand_dims(datas[i],axis=0)

defaultdict(<function __main__.<lambda>>,
            {'<PAD>': 1,
             'lc': 2,
             'worst': 3,
             'imaginarypeach': 4,
             'tagalog': 5,
             'naw': 6,
             'funny': 7,
             'guilt': 8,
             'mang': 9,
             'lurk': 10,
             'bear': 11,
             'nap': 12,
             'child': 13,
             '771ukx4zhzq': 14,
             'serv': 15,
             'sof': 16,
             'tid': 17,
             'premy': 18,
             'long': 19,
             'plug': 20,
             'win': 21,
             'intend': 22,
             'bout': 23,
             'acc': 24,
             'swap': 25,
             'sunday': 26,
             'dizzy': 27,
             'jennma': 28,
             'supersport': 29,
             'york': 30,
             'ceprv': 31,
             'downtown': 32,
             'sel': 33,
             'biolog': 34,
             'magneto': 35,
             'autograph': 36,
             'rofl': 3